### Create Directories to Store Images Based on the Keras Preprocessing Directory Format

In [ ]:
!mkdir /content/data

In [ ]:
!mkdir /content/data/train_image
!mkdir /content/data/test_image

In [ ]:
%cd /content/data/train_image
%cp -av /content/drive/MyDrive/FloodNet_Track2/Images/Train_Image Train_Image

In [ ]:
%cd /content/data/test_image
%cp -av /content/drive/MyDrive/FloodNet_Track2/Images/Test_Image test_Image

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import json
from pathlib import Path
import regex as re
import matplotlib.pyplot as plt
import os

import spacy
import scipy.io
import gc
from sklearn.preprocessing import LabelEncoder
import pickle
from PIL import Image

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Reshape, Flatten
from keras import Input
from keras.layers.recurrent import LSTM
from keras.layers import concatenate
from keras.layers.merge import Concatenate
from keras.models import model_from_json, Model
from keras.utils import plot_model
from collections import defaultdict
import operator
from keras.utils import np_utils, generic_utils
from progressbar import Bar, ETA, Percentage, ProgressBar
from itertools import zip_longest
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory, text_dataset_from_directory
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
BATCH_SIZE = 4
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory("/content/data/train_image",
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
test_dataset = image_dataset_from_directory("/content/data/test_image",
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 1448 files belonging to 1 classes.
Found 450 files belonging to 1 classes.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Define Layers in the Model

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)

### Import VGG16

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  19


### Freeze Specific Layers for Training

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 11

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

for layer in base_model.layers:
  sp = '   '[len(layer.name):]
  print(layer.name, sp, layer.trainable)

input_1  False
block1_conv1  False
block1_conv2  False
block1_pool  False
block2_conv1  False
block2_conv2  False
block2_pool  False
block3_conv1  False
block3_conv2  False
block3_conv3  False
block3_pool  False
block4_conv1  True
block4_conv2  True
block4_conv3  True
block4_pool  True
block5_conv1  True
block5_conv2  True
block5_conv3  True
block5_pool  True


### Define Model

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0     

Training

In [ ]:
initial_epochs = 20

save_model = tf.keras.callbacks.ModelCheckpoint("/content/drive/Weights/vgg16.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(
    train_dataset,
    epochs = initial_epochs,
    validation_data = validation_dataset,
    callbacks = [save_model]
)

Epoch 1/20
362/362 [==============================] - 278s 763ms/step - loss: 1.6193e-12 - accuracy: 1.0000 - val_loss: 4.9999e-14 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to /content/drive/MyDrive/FloodNet Competition Notebooks/Weights/vgg16.h5
Epoch 2/20
362/362 [==============================] - 264s 716ms/step - loss: 7.5814e-12 - accuracy: 1.0000 - val_loss: 4.7437e-14 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/20
362/362 [==============================] - 263s 713ms/step - loss: 5.3637e-13 - accuracy: 1.0000 - val_loss: 4.7285e-14 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/20
362/362 [==============================] - 262s 711ms/step - loss: 1.0235e-12 - accuracy: 1.0000 - val_loss: 4.6877e-14 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/20
362/362 [==============================] - 263s 712ms/step -

### Modifying Model for Feature Extraction

In [ ]:
import keras
model = Sequential()
reconstructed_model = keras.models.load_model("/content/drive/Weights/vgg16.h5")
for layer in reconstructed_model.layers[:5]: # go through until last layer
    model = layer
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
import keras
reconstructed_model = keras.models.load_model("/content/drive/Weights/vgg16.h5")
ip = Input([224, 224, 3])
x = preprocess_input(ip)
Model = Sequential()
for layer in base_model.layers[:5]: 
    Model = layer

model1 = Sequential()
model1.add(Model)
model1.add(Flatten())
model1.add(Dense(4096))
x = model1(x, training=True)

model_ = tf.keras.Model(inputs=[ip], outputs=[x])
model_.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_2 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 4096)              117479232 
Total params: 117,479,232
Trainable params: 115,743,744
Non-trainable params: 1,735,488
_________________________________________________________________


### Extract Image Features


In [ ]:
image_list_train = []
image_list_test = []
for images, labels in train_dataset:
  image_list_train.append(images)

for images, labels in test_dataset:
  image_list_test.append(images) 

In [ ]:
image_features_train = []
image_features_test = []
for image in image_list_train:
  image_features_train.append(model_(image))

for image in image_list_test:
  image_features_test.append(model_(image))

In [ ]:
train_img_features = np.transpose(np.vstack(image_features_train))
test_img_features = np.transpose(np.vstack(image_features_test))
print(test_img_features.shape)

(4096, 450)


### Store Image Features in the Required Format


In [ ]:
import pickle
features_train = open('/content/drive/Feature_Files/vgg_features_train.txt', 'wb')
pickle.dump(train_img_features, features_train)
features_train.close()

features_test = open('/content/drive/Feature_Files/vgg_features_test.txt', 'wb')
pickle.dump(test_img_features, features_test)
features_test.close()

In [ ]:
features = open('/content/drive/Feature_Files/vgg_features_test.txt',"rb")
imgfam = pickle.load(features)
print(imgfam.shape)

(4096, 450)
